In [ ]:
import pandas as pd
import altair as alt
import sqlite3
import os, json
import evaluate, meta, modeling, data_wrangling

In [ ]:
tf_root = '/home/jupyter/tf'
batch_name = "error_injection_timing_test"
batch_output_dir = os.path.join(tf_root, "models", "batch_run", batch_name)

In [ ]:
# Load batch config from json
code_name = 'high_stress_long_pretrain'
batch_json_file = os.path.join(batch_output_dir, "batch_config.json")
with open(batch_json_file) as f:
    batch_cfgs = json.load(f)

In [ ]:
# Load model
i = 0
model_json_file = os.path.join(tf_root, batch_cfgs[i]['model_folder'], 'model_config.json')
cfg = meta.ModelConfig.from_json(model_json_file)
model = modeling.HS04Model(cfg)
data = data_wrangling.MyData()

In [ ]:
from importlib import reload
reload(evaluate)

In [ ]:
oral_eval = evaluate.EvalOral(cfg, model, data)
oral_eval.tmp_eval_ss()

In [ ]:
ss_csv = os.path.join(cfg.path['model_folder'], 'eval', 'train_ss_item_df.csv')
df = pd.read_csv(ss_csv, index_col=0)
df['word'] = df.item

In [ ]:
from ipywidgets import interact, widgets

In [ ]:
def get_n_features(word):
    if word in data.testsets['train']['item']:
        idx = data.testsets['train']['item'].index(word)
        semantic_vector = data.testsets['train']['sem'][idx,:]
        return(sum(semantic_vector)) 

In [ ]:
img_csv = os.path.join(tf_root, "issues", "preprocessing", "cortese2004norms.csv")
cortese = pd.read_csv(img_csv, skiprows=9, na_filter=False)
cortese = cortese[["item", "rating"]]
cortese = cortese.rename({"item": "word", "rating":"img"}, axis=1)
cortese["n_features"] = cortese.word.apply(get_n_features)
cortese = cortese.merge(data.df_train[['word', 'wf']])
cortese.to_csv('cortese_compiled.csv')

In [ ]:
cortese

In [ ]:
import altair as alt

def plot_img_vs_sse(df, epoch, timetick):
    acc_sse_now = df.loc[(df.epoch == epoch) & (df.timetick == timetick), ["word", "sse", "acc"]]
    df = cortese.merge(acc_sse_now, 'left')
    
    plot = alt.Chart(df).mark_point().encode(x="img", y="sse", tooltip=["word", "img"])
    plot = plot + plot.transform_regression('img', 'sse').mark_line(color="red")
    plot = plot.properties(title="sse vs. img")
    
    print(df.corr())
    print(df)
    return(plot)
#     p = alt.Chart(df).mark_point().encode(x="img", y="sse", tooltip=["word", "img"])
#     p += p.transform_regression('img', 'sse').mark_line(color="red")
#     p.properties(title = cortese.corr())

In [ ]:
plot_img_vs_sse(df, 100, 12)

In [ ]:
plot_df = df.groupby(['epoch', 'timetick']).mean().reset_index()

In [ ]:
# Overall semantic attractor accuracy

alt.Chart(plot_df).mark_rect().encode(x="epoch:O", y="timetick:O", color="acc")

In [ ]:
interact()

In [ ]:
meta.csv_to_bigquery(, dataset_name="triangle_oral", table_name="ss")